主要就是，扩大了 maxlen 。表现就能提高很多。

In [1]:
import sys
from kaitoupao import *

storage dir: /Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification
code dir: /Users/minkexiu/Documents/GitHub/Tianchi_NLPNewsClassification 

06 20 03
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：6 本卦下：4 变爻：3


,水雷屯,山地剥,水火既济
上卦,☵坎水,☶艮土,☵坎水
下卦,☳震木,☷坤土,☲离火


12 07 11 戌时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：4 本卦下：7 变爻：5


,雷山小过,泽风大过,泽山咸
上卦,☳震木,☱兑金,☱兑金
下卦,☶艮土,☴巽木,☶艮土


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import f1_score

In [3]:
device = torch.device("cpu")

# 加载训练集和测试集，将全量字符列表给它弄出来

In [52]:
data_train = pd.read_csv(create_originalData_path("train_set.csv"), sep="\t").sample(10000).reset_index(drop=True)
data_test = pd.read_csv(create_originalData_path("test_a.csv"), sep="\t")

In [53]:
data_train["data_id"] = range(data_train.shape[0])
data_test["data_id"] = range(data_test.shape[0])
data_test["label"] = -1

In [54]:
data_train.shape, data_test.shape

((10000, 3), (50000, 3))

In [55]:
type_of_class = data_train.label.nunique()

In [56]:
def get_text_len(txt):
    return len(txt.strip().split())
data_train["txt_len"] = data_train.text.progress_apply(get_text_len)
data_test["txt_len"] = data_test.text.progress_apply(get_text_len)

100%|██████████████████████████████████| 50000/50000 [00:00<00:00, 64301.14it/s]


In [57]:
data_train = data_train.sort_values("txt_len").reset_index(drop=True)
data_test = data_test.sort_values("txt_len").reset_index(drop=True)

In [58]:
data_train

,label,text,data_id,txt_len
0,10,83 3961 5445 5139 5705 7404 6357 2614 3015 446...,6186,14
1,8,6616 3740 3433 1219 781 1402 6905 5474 6616 37...,1509,16
2,0,5656 2304 6289 4464 3370 4149 6065 2465 6469 6...,6886,18
3,1,1580 3568 7539 4853 5816 517 1724 6832 6854 21...,9528,19
4,1,1580 3568 7539 6875 1402 340 4063 6654 3469 54...,8488,20
...,...,...,...,...
9995,6,3659 3370 4464 3370 1519 3870 3641 6248 2555 3...,7497,10302
9996,0,1853 4643 5498 151 1168 5718 2465 2400 6899 66...,2433,10833
9997,1,5036 6501 2899 4163 2465 2799 2073 3659 1324 4...,9926,14531
9998,1,4151 4853 299 6357 151 2859 62 3300 5491 3659 ...,3627,15266


In [59]:
data_test

,text,data_id,label,txt_len
0,3366 6407 932 2848 2400 3870 3242 2685 4490 42...,5081,-1,14
1,1141 473 1407 4802 7539 3961 1227 2380 5689 68...,39409,-1,14
2,2289 6357 2109 4139 248 3193 4595 1148 3397 30...,649,-1,14
3,3945 7444 2109 4139 5122 4148 2490 4811 5612 3...,22985,-1,15
4,5041 1877 1241 5702 736 2396 2465 2210 6040 22...,18530,-1,17
...,...,...,...,...
49995,299 6357 151 2859 1866 7495 3961 1227 5860 121...,5558,-1,38588
49996,3700 2073 4464 3700 4853 299 6357 151 2859 584...,9236,-1,38906
49997,4646 2073 4464 4853 299 6357 151 2859 584 1920...,19746,-1,38930
49998,4516 5264 299 6357 151 2859 62 3300 5491 4464 ...,15053,-1,41459


# 定义网络结构

In [97]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [98]:
class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        
        self.lstm_emb = nn.LSTM(input_size=embed_dim, hidden_size=embed_dim, num_layers=2, batch_first=True)
        # self.gru_emb = nn.GRU(input_size=embed_dim, hidden_size=embed_dim, num_layers=2, batch_first=True)
        
        self.pos_emb = nn.Embedding(5, embed_dim) ## 这个10是根据 self.lstm_emb，self.gru_emb 算出来的。

    def forward(self, x):
        emb_here = torch.squeeze(
            self.token_emb(x)
        )
        
        out1, _1 = self.lstm_emb(emb_here)
        lstm_part = torch.cat([
            _1[0].permute(1, 0, 2), ## h_t；总共2层，层数取决于初始化的层数。
            _1[1].permute(1, 0, 2), ## c_t；总共2层，层数取决于初始化的层数。
            torch.mean(out1, dim=1).unsqueeze(1) ## output，把每一个输入对应的输出，给它平均一下。共1层。
        ], dim=1)
        ## there are 5 seq_lens. 

        # out2, _2 = self.gru_emb(emb_here)
        # gru_part = torch.cat([
        #     _2[0].permute(1, 0, 2), 
        #     _2[1].permute(1, 0, 2),
        #     torch.mean(out2, dim=1).unsqueeze(1)
        # ], dim=1)
        # ## there are 5 seq_lens. 

        seq_part = lstm_part
        # torch.cat([
        #     lstm_part, gru_part
        # ], dim=1)
        # ## in total, there are 5 seq_lens. 

        positions = torch.arange(0, 5, device=x.device).unsqueeze(0).expand(x.shape[0], 5)
        
        return seq_part + self.pos_emb(positions)

In [99]:
# 定义模型
class TransformerModel(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.Linear(embed_dim, 20)
        self.dropout2 = nn.Dropout(0.1)
        self.dense2 = nn.Linear(20, type_of_class)
        
    def forward(self, x):
        # print(x)
        x = self.embedding_layer(x).transpose(0, 1)  # Transformer expects (seq_len, batch_size, embed_dim)
        x = self.transformer_block(x)
        x = x.transpose(0, 1)
        x = self.global_avg_pool(x.permute(0, 2, 1)).squeeze(-1)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        x = self.dropout2(x)
        return F.log_softmax(self.dense2(x), dim=-1)

# 准备数据集

In [63]:
## 【TODO】
## https://www.cnblogs.com/picassooo/p/13577527.html 尝试一下 https://blog.csdn.net/xinjieyuan/article/details/108562360

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
train_ori, valid_ori = train_test_split(data_train, test_size=0.3)

In [66]:
train_ori.reset_index(drop=True, inplace=True)
valid_ori.reset_index(drop=True, inplace=True)

In [67]:
# 下载数据并进行预处理
vocab_size = 8000  # 只考虑前 20k 词汇
maxlen = 800  # 只考虑每条评论的前 200 个词
batchsize = 16

In [68]:
def preprocess_seq_str_2_int(seq, len_lim = maxlen):
    rst = [int(wd) for idx, wd in enumerate(seq.strip().split())]
    return rst

In [69]:
x_train = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in train_ori.text]
y_train = torch.tensor(list(train_ori.label), dtype=torch.long)
id_train = torch.tensor(list(train_ori.data_id), dtype=torch.long)

In [70]:
x_valid = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in valid_ori.text]
y_valid = torch.tensor(list(valid_ori.label), dtype=torch.long)
id_valid = torch.tensor(list(valid_ori.data_id), dtype=torch.long)

In [71]:
class MyData(Dataset):
    def __init__(self, data, label, data_id):
        self.data = data
        self.label = label
        self.data_id = data_id
 
    def __len__(self):
        return len(self.data)
 
    def __getitem__(self, idx):
        tuple_ = (self.data[idx], self.label[idx], self.data_id[idx])
        return tuple_

In [72]:
def collate_fn(data_tuple):   # data_tuple是一个列表，列表中包含batchsize个元组，每个元组中包含数据和标签
    data_tuple.sort(key=lambda x: len(x[0]), reverse=True)
    data = [sq[0] for sq in data_tuple]
    label = [sq[1] for sq in data_tuple]
    data_id = [sq[2] for sq in data_tuple]
    data_length = [len(sq) for sq in data]
    data = pad_sequence(data, batch_first=True, padding_value=0.0)     # 用零补充，使长度对齐
    label = torch.tensor(label, dtype=torch.long) # pad_sequence(label, batch_first=True, padding_value=0.0)   # 这行代码只是为了把列表变为tensor
    return data.unsqueeze(-1), label, data_length, data_id

In [73]:
train_dataset = MyData(x_train, y_train, id_train)
val_dataset = MyData(x_valid, y_valid, id_valid)

In [74]:
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=True, collate_fn=collate_fn)

In [75]:
x_oot = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in data_test.text]
y_oot = torch.tensor(list(data_test.label), dtype=torch.long)
id_oot = torch.tensor(list(data_test.data_id), dtype=torch.long)

In [76]:
oot_dataset = MyData(x_oot, y_oot, id_oot)
oot_loader = DataLoader(oot_dataset, batch_size=batchsize, shuffle=True, collate_fn=collate_fn)

In [83]:
# for x in train_loader:
#     break

# 构建模型以及训练

In [100]:
model = TransformerModel(maxlen, vocab_size, embed_dim=8, num_heads=2, ff_dim=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [101]:
model

TransformerModel(
  (embedding_layer): TokenAndPositionEmbedding(
    (token_emb): Embedding(8000, 8)
    (lstm_emb): LSTM(8, 8, num_layers=2, batch_first=True)
    (pos_emb): Embedding(5, 8)
  )
  (transformer_block): TransformerBlock(
    (att): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (ffn): Sequential(
      (0): Linear(in_features=8, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=8, bias=True)
    )
    (layernorm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (layernorm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dense1): Linear(in_features=8, out_features=20, bias=True)
  (dropout2): Dropout(p=0.1, inplace=False)
  (dense2): Linear(in_fea

In [108]:
# 训练和评估模型
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        for inputs, targets, d_l, d_id in tqdm.tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%')
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            total_predicted = []
            total_label = []
            for inputs, targets, d_l, d_id in tqdm.tqdm(val_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                total_predicted += list(predicted)
                total_label += list(targets)
        f1 = f1_score(total_label, total_predicted, average='macro')
                
        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%, f1 score is {f1}')

In [109]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

100%|█████████████████████████████████████████| 438/438 [02:26<00:00,  2.99it/s]


Epoch 1/1, Loss: 2.2930872690732076, Accuracy: 18.585714285714285%


100%|█████████████████████████████████████████| 188/188 [00:13<00:00, 13.94it/s]

Validation Loss: 2.26965553646392, Accuracy: 18.333333333333332%, f1 score is 0.022132796780684104


In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=3)

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

In [ ]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

# 预测一下试试

In [ ]:
with torch.no_grad():
    total_predicted = []
    for inputs, targets, d_l, d_id in tqdm.tqdm(oot_loader):
        outputs = model(inputs[0])
        _, predicted = torch.max(outputs, 1)
        total_predicted += list(predicted)

In [ ]:
oot_rst = [int(x) for x in total_predicted]
len(oot_rst)

In [ ]:
store_data_to_newbasepath(pd.DataFrame({"label": oot_rst}), "rst-20250104_2", fmt="csv")